In [2]:
import pandas as pd


In [7]:
data = pd.read_csv('data/acled_all.csv')

/Users/crismacg/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
data.head()

,Unnamed: 0,ISO,EVENT_ID_CNTY,EVENT_ID_NO_CNTY,EVENT_DATE,YEAR,TIME_PRECISION,EVENT_TYPE,ACTOR1,ASSOC_ACTOR_1,...,LOCATION,LATITUDE,LONGITUDE,GEO_PRECISION,SOURCE,SOURCE_SCALE,NOTES,FATALITIES,TIMESTAMP,CONTINENT
0,1,12,ALG1,1.0,01-January-1997,1997,1,Violence against civilians,GIA: Armed Islamic Group,NaN,...,Douaouda,36.672,2.789,1,www.algeria-watch.org,National,5 January: Beheading of 5 citizens in Douaouda...,5,1546559533,Africa
1,2,12,ALG2,2.0,02-January-1997,1997,1,Violence against civilians,GIA: Armed Islamic Group,NaN,...,Hassasna,36.133,0.883,1,www.algeria-watch.org,National,Two citizens were beheaded in Hassasna.,2,1546559533,Africa
2,3,12,ALG3,3.0,03-January-1997,1997,1,Violence against civilians,GIA: Armed Islamic Group,NaN,...,Hassi El Abed,34.966,-0.290,1,www.algeria-watch.org,National,Two citizens were killed in a raid on the vill...,2,1546559528,Africa
3,4,12,ALG4,4.0,04-January-1997,1997,1,Violence against civilians,GIA: Armed Islamic Group,NaN,...,Blida,36.469,2.829,1,www.algeria-watch.org,National,4 January: 16 citizens were murdered in the vi...,16,1546559533,Africa
4,5,12,ALG5,5.0,05-January-1997,1997,1,Violence against civilians,GIA: Armed Islamic Group,NaN,...,Douaouda,36.672,2.789,1,www.algeria-watch.org,National,5 January: Killing of 18 citizens in the Olivi...,18,1546559533,Africa


In [62]:
import torch
import torch.utils.data as tud
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter, defaultdict
import operator
import os, math
import numpy as np
import random
import copy

# Feel free to define your own word_tokenizer instead of this naive 
# implementation. You may also use word_tokenize from nltk library 
# (from nltk import word_tokenize), which works better but slower. 
def remove_stopwords(l):
    STOP  = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'] 
    l_clean = []
    for i in l: 
        if i not in STOP:
            l_clean.append(i)
    return l_clean 


def word_tokenize(s, clean = False):
    if type(s) != str: 
        return None
    split_l = s.lower().replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('!', '').replace('?', '').split()
    if clean: 
        clean_l = remove_stopwords(split_l)
        return clean_l
    
    return split_l
    
# set the random seeds so the experiments can be replicated exactly
seed = 30255
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [56]:
VOCAB_SIZE = 5000
class Model:
    def __init__(self, data, clean = False):
        # Vocabulary is a set that stores every word seen in the 
        # training data
        self.vocab_counts = Counter([word for content in data 
                              for word in word_tokenize(content, clean) if word]
                            ).most_common(VOCAB_SIZE-1)
        # word to index mapping
        self.word_to_idx = {k[0]: v+1 for v, k in 
                            enumerate(self.vocab_counts)}
        # all the unknown words will be mapped to index 0
        self.word_to_idx["UNK"] = 0 
        self.vocab = set(self.word_to_idx.keys())


In [57]:
class TextClassificationDataset(tud.Dataset):
    '''
    PyTorch provides a common dataset interface. 
    See https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    The dataset encodes documents into indices. 
    With the PyTorch dataloader, you can easily get batched data for 
    training and evaluation. 
    '''
    def __init__(self, word_to_idx, data):
        
        self.data = data
        self.word_to_idx = word_to_idx
        self.vocab_size = VOCAB_SIZE
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = np.zeros(self.vocab_size)
        item = torch.from_numpy(item)
        for word in word_tokenize(self.data[idx]):
            item[self.word_to_idx.get(word, 0)] += 1
        return item

In [58]:
data.dropna(subset=['NOTES'], inplace = True)
x_wstopwords = Model(data['NOTES'], False)

In [52]:
x_wstopwords.vocab_counts

[('the', 677693),
 ('in', 492704),
 ('of', 456125),
 ('and', 324766),
 ('a', 293996),
 ('on', 230236),
 ('to', 166074),
 ('were', 143442),
 ('forces', 98638),
 ('by', 93749),
 ('killed', 87105),
 ('at', 85940),
 ('was', 80448),
 ('reported', 73558),
 ('protest', 73046),
 ('against', 73045),
 ('an', 66592),
 ('with', 63296),
 ('police', 60726),
 ('area', 54486),
 ('district', 54456),
 ('for', 53696),
 ('from', 49916),
 ('no', 49861),
 ('as', 48861),
 ('two', 45376),
 ('fatalities', 45082),
 ('staged', 41551),
 ('their', 40826),
 ('city', 39068),
 ('al', 39049),
 ('injured', 38883),
 ('between', 38514),
 ('people', 35799),
 ('militants', 34736),
 ('village', 34615),
 ('that', 32411),
 ('town', 32373),
 ('clashes', 30781),
 ('is', 30445),
 ('attack', 30374),
 ('over', 29499),
 ('state', 29199),
 ('military', 29055),
 ('government', 28915),
 ('after', 28040),
 ('near', 27502),
 ('regime', 26724),
 ('one', 25975),
 ('they', 25948),
 ('province', 25025),
 ('during', 24579),
 ('members', 2455

In [63]:
x_nstopwords = Model(data['NOTES'], True)


In [65]:
x_nstopwords.vocab_counts

[('forces', 98638),
 ('killed', 87105),
 ('reported', 73558),
 ('protest', 73046),
 ('police', 60726),
 ('area', 54486),
 ('district', 54456),
 ('two', 45376),
 ('fatalities', 45082),
 ('staged', 41551),
 ('city', 39068),
 ('al', 39049),
 ('injured', 38883),
 ('people', 35799),
 ('militants', 34736),
 ('village', 34615),
 ('town', 32373),
 ('clashes', 30781),
 ('attack', 30374),
 ('state', 29199),
 ('military', 29055),
 ('government', 28915),
 ('near', 27502),
 ('regime', 26724),
 ('one', 25975),
 ('province', 25025),
 ('members', 24558),
 ('2', 23635),
 ('group', 23375),
 ('soldiers', 23189),
 ('casualties', 22737),
 ('attacked', 22437),
 ('fighters', 22384),
 ('army', 22326),
 ('demonstration', 21403),
 ('security', 21236),
 ('injuries', 21214),
 ('unknown', 20873),
 ('armed', 20695),
 ('killing', 20484),
 ('civilians', 20206),
 ('unidentified', 19889),
 ('three', 19835),
 ('took', 19755),
 ('coalition', 19469),
 ('areas', 18315),
 ('several', 18077),
 ('10', 17525),
 ('protested', 1